In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
train_dir = 'D:\datasets\Cataract Data\Train'
test_dir = 'D:\datasets\Cataract Data\Test'
input_shape = (128, 128)

### Preparing the data

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, shear_range=0.2, zoom_range=0.2)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=input_shape, batch_size=16, class_mode='categorical')

In [ ]:
test_generator = test_datagen.flow_from_directory(test_dir, target_size=input_shape, batch_size=16, class_mode='categorical')

In [ ]:
num_classes = len(train_generator.class_indices)

### Building MEDNet Model

In [ ]:
def build_mednet(input_shape, num_classes):
    model = keras.Sequential()

    # Traditional Convolutional Layers
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (5, 5), activation='relu'))
    model.add(layers.BatchNormalization())

    # Parallel Convolutional Blocks
    for _ in range(8):
        for kernel_size in [1, 3, 5, 7]:
            model.add(layers.Conv2D(64, (kernel_size, kernel_size), activation='relu'))
        model.add(layers.BatchNormalization())

    # Connections between Blocks
    for _ in range(12):
        model.add(layers.Conv2D(64, (1, 1), activation='relu'))

    # Fully Connected Layers with Dropout
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [ ]:
mednet_model = build_mednet(input_shape + (3,), num_classes)

In [ ]:
mednet_model.summary()

In [ ]:
mednet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the model

In [ ]:
history = mednet_model.fit(train_generator, validation_data=test_generator, epochs=1)

In [ ]:
test_loss, test_accuracy = mednet_model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")